<a href="https://colab.research.google.com/github/pseudope/2021_F_Level2/blob/main/Projects/2%EC%A1%B0/3i4k_1008_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy
!pip install soynlp

     |████████████████████████████████| 19.4 MB 5.4 MB/s 
     |████████████████████████████████| 448 kB 44.0 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
     |████████████████████████████████| 416 kB 13.1 MB/s 


In [2]:
import pandas as pd
import numpy as np

import konlpy
from konlpy.tag import Okt
from keras.preprocessing.text import Tokenizer

from keras.layers import Embedding, Dense, LSTM, Bidirectional, Dropout
from keras.models import Sequential 
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

### 데이터 다운로드

---

In [3]:
!wget https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_train_val.txt
!wget https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_test.txt

--2021-10-08 08:26:07--  https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_train_val.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2661060 (2.5M) [text/plain]
Saving to: ‘fci_train_val.txt’

fci_train_val.txt   100%[===================>]   2.54M  --.-KB/s    in 0.03s   

2021-10-08 08:26:08 (83.9 MB/s) - ‘fci_train_val.txt’ saved [2661060/2661060]

--2021-10-08 08:26:08--  https://raw.githubusercontent.com/warnikchow/3i4k/master/data/train_val_test/fci_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 

In [4]:
#데이터 불러오기

def read_data(filename):

    with open(filename, "r", encoding="UTF8") as f:
        data = [line.split("\t") for line in f.read().splitlines()]

    return data

In [5]:

train_data = read_data("/content/fci_train_val.txt")


#X,y
X_train_data = [t[1] for t in train_data]
y_train_data = [int(t[0]) for t in train_data]



test_data = read_data("/content/fci_test.txt")


#X,y
X_test_data = [t[1] for t in test_data]
y_test_data = [int(t[0]) for t in test_data]

In [6]:
print(len(X_train_data))
print(len(y_train_data))
print(len(X_test_data))
print(len(y_test_data))

55134
55134
6121
6121


In [7]:
#불용어 사전 가져오기

df = pd.read_csv('https://raw.githubusercontent.com/cranberryai/todak_todak_python/master/machine_learning_text/clean_korean_documents/korean_stopwords.txt', header=None)
df[0] = df[0].apply(lambda x: x.strip())

stopwords = df[0].to_numpy()

print(stopwords)
print(len(stopwords))

['이' '있' '하' '것' '들' '그' '되' '수' '이' '보' '않' '없' '나' '사람' '주' '아니' '등' '같'
 '우리' '때' '년' '가' '한' '지' '대하' '오' '말' '일' '그렇' '위하' '때문' '그것' '두' '말하'
 '알' '그러나' '받' '못하' '일' '그런' '또' '문제' '더' '사회' '많' '그리고' '좋' '크' '따르' '중'
 '나오' '가지' '씨' '시키' '만들' '지금' '생각하' '그러' '속' '하나' '집' '살' '모르' '적' '월' '데'
 '자신' '안' '어떤' '내' '경우' '명' '생각' '시간' '그녀' '다시' '이런' '앞' '보이' '번' '나' '다른'
 '어떻' '여자' '개' '들' '사실' '이렇' '점' '싶' '말' '정도' '좀' '원' '잘' '통하' '소리' '놓']
98


### 02. 토큰화 및 인코딩 (augmentation 생략)

In [8]:
okt = Okt()

#형태소 토큰화


X_train_new=[]
for sentence_new in X_train_data : 
    temp_X = [] 
    temp_X = okt.morphs(sentence_new, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] #불용어에 해당하지 않는 토큰만 저장
    X_train_new.append(temp_X)

X_test_new = [] 
for sentence_new in X_test_data : 
    temp_X = [] 
    temp_X = okt.morphs(sentence_new, stem=True) # 토큰화 
    temp_X = [word for word in temp_X if not word in stopwords] #불용어에 해당하지 않는 토큰만 저장
    X_test_new.append(temp_X)

In [9]:
print(X_train_new[:5])
print(X_test_new[:5])

[['만화'], ['이치'], ['약'], ['사이'], ['짜다']]
[['연두색'], ['되다'], [], ['그걸루'], ['연구소']]


In [10]:
#정수 인코딩

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_new)

total_cnt = len(tokenizer.word_index)
vocab_size = total_cnt + 1

In [11]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train_new)
X_train = tokenizer.texts_to_sequences(X_train_new)
X_test = tokenizer.texts_to_sequences(X_test_new)

In [12]:
X_train[:5]

[[3906], [7779], [389], [568], [915]]

In [13]:
#15로 패딩

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 15
# 한국어 특성상 문장 끝이 중요하므로 오른쪽에 치우치게 패딩

X_train_pad_new = pad_sequences(X_train, maxlen = max_len)
X_test_pad_new = pad_sequences(X_test, maxlen = max_len)

print(X_train_pad_new)
print(len(X_train_pad_new))
print(len(X_test_pad_new))

[[   0    0    0 ...    0    0 3906]
 [   0    0    0 ...    0    0 7779]
 [   0    0    0 ...    0    0  389]
 ...
 [   0    0    0 ...    0  317   32]
 [   0    0    0 ...    0   20   66]
 [   0    0    0 ... 1075  110    1]]
55134
6121


In [14]:
from tensorflow.keras.utils import to_categorical
print(len(set(y_train_data)))
y_train = to_categorical(y_train_data,len(set(y_train_data)))
y_test = to_categorical(y_test_data,len(set(y_train_data)))


print(len(X_train_pad_new))
print(len(X_test_pad_new))
print(len(y_train))
print(len(y_test))

7
55134
6121
55134
6121


In [15]:
from sklearn.model_selection import train_test_split

X_train_new, X_valid_new, y_train_new, y_valid_new = train_test_split(X_train_pad_new, y_train, test_size=0.1, shuffle=True, stratify=y_train)
X_test_new = X_test_pad_new
y_test_new = y_test
print(y_valid_new[0])
print(len(X_valid_new))
print(len(y_valid_new))

[0. 0. 1. 0. 0. 0. 0.]
5514
5514


In [16]:
from sklearn.model_selection import train_test_split

X_train_new, X_valid_new, y_train_new, y_valid_new = train_test_split(X_train_pad_new, y_train, test_size=0.1, shuffle=True, stratify=y_train)
X_test_new = X_test_pad_new
y_test_new = y_test


In [17]:
print(X_train_new.shape)
print(X_test_new.shape)
print(X_valid_new.shape)
print(y_train_new.shape)
print(y_test_new.shape)
print(y_valid_new.shape)

(49620, 15)
(6121, 15)
(5514, 15)
(49620, 7)
(6121, 7)
(5514, 7)


## Modeling 1

In [18]:
!pip install tensorflow-addons

from tensorflow_addons.metrics import F1Score

     |████████████████████████████████| 1.1 MB 13.1 MB/s 


In [19]:
f1 = F1Score(num_classes=7, name="F1")
weighted_f1 = F1Score(num_classes=7, average="weighted", name="Weighted_F1")

In [20]:
#pretrained data 다운로드 후 사용 (코랩 용)
!pip install gensim
!pip install gdown

In [21]:
!gdown --id 1jHbjOcnaLourFzNuP47yGQVhBTq6Wgor --output vector.zip

Downloading...
From: https://drive.google.com/uc?id=1jHbjOcnaLourFzNuP47yGQVhBTq6Wgor
To: /content/vector.zip
100% 874M/874M [00:10<00:00, 80.1MB/s]


In [22]:
!unzip -qq "/content/vector.zip"

In [23]:
from gensim.models.fasttext import FastText as FT_gensim
loaded_model = FT_gensim.load_fasttext_format('/content/model_drama.bin')
embedding_matrix = np.zeros((vocab_size, 100))  
for word, i in tokenizer.word_index.items():
    if word in loaded_model:
        embedding_vector = loaded_model[word]
    else:
        continue     
    embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [24]:
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.models import Model

In [25]:
max_features = vocab_size
maxlen =15
embed_size = 100

In [61]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [63]:
def get_model(): #GRU 기반 모델, kaggle의 toxic classification 코드에서 따옴
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.5)(x)
    x = Bidirectional(GRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(7, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["acc", f1, weighted_f1])

    return model

In [67]:
model=get_model()
history = model.fit(X_train_new, y_train_new, 
                    epochs=50, callbacks=[es, mc], batch_size=100, validation_data = (X_valid_new, y_valid_new))

Epoch 1/50
497/497 [==============================] - 13s 18ms/step - loss: 0.9677 - acc: 0.6580 - F1: 0.4876 - Weighted_F1: 0.6523 - val_loss: 0.7215 - val_acc: 0.7534 - val_F1: 0.5702 - val_Weighted_F1: 0.7363

Epoch 00001: val_loss did not improve from 0.52624
Epoch 2/50
497/497 [==============================] - 8s 16ms/step - loss: 0.6942 - acc: 0.7671 - F1: 0.5862 - Weighted_F1: 0.7529 - val_loss: 0.6550 - val_acc: 0.7809 - val_F1: 0.6217 - val_Weighted_F1: 0.7699

Epoch 00002: val_loss did not improve from 0.52624
Epoch 3/50
497/497 [==============================] - 8s 16ms/step - loss: 0.6189 - acc: 0.7900 - F1: 0.6445 - Weighted_F1: 0.7803 - val_loss: 0.5980 - val_acc: 0.8009 - val_F1: 0.6887 - val_Weighted_F1: 0.7979

Epoch 00003: val_loss did not improve from 0.52624
Epoch 4/50
497/497 [==============================] - 8s 15ms/step - loss: 0.5759 - acc: 0.8031 - F1: 0.6809 - Weighted_F1: 0.7959 - val_loss: 0.5614 - val_acc: 0.8070 - val_F1: 0.6916 - val_Weighted_F1: 0.8002

In [68]:
loaded_model = load_model('best_model.h5')

In [69]:
loaded_model.evaluate(X_test_new, y_test_new)

192/192 [==============================] - 2s 6ms/step - loss: 0.5184 - acc: 0.8218 - F1: 0.7370 - Weighted_F1: 0.8181


[0.5183678865432739,
 0.8217611312866211,
 array([0.9214403 , 0.8142857 , 0.88165516, 0.81100196, 0.50871074,
        0.68085104, 0.54117644], dtype=float32),
 0.8181202411651611]

In [31]:
def get_model_two():
    model = Sequential()
    model.add(Embedding(max_features,128))
    model.add(LSTM(128))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))    
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["acc", f1, weighted_f1])
    return model

In [32]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [33]:
model=get_model_two()
history = model.fit(X_train_new, y_train_new, 
                    epochs=50, callbacks=[es, mc], batch_size=100, validation_data = (X_valid_new, y_valid_new))

Epoch 1/50
497/497 [==============================] - 9s 13ms/step - loss: 0.8371 - acc: 0.7119 - F1: 0.5362 - Weighted_F1: 0.7048 - val_loss: 0.6232 - val_acc: 0.7864 - val_F1: 0.6164 - val_Weighted_F1: 0.7748

Epoch 00001: val_acc improved from -inf to 0.78636, saving model to best_model.h5
Epoch 2/50
497/497 [==============================] - 6s 12ms/step - loss: 0.5711 - acc: 0.8070 - F1: 0.6827 - Weighted_F1: 0.8003 - val_loss: 0.5772 - val_acc: 0.7962 - val_F1: 0.7006 - val_Weighted_F1: 0.7921

Epoch 00002: val_acc improved from 0.78636 to 0.79616, saving model to best_model.h5
Epoch 3/50
497/497 [==============================] - 6s 12ms/step - loss: 0.5000 - acc: 0.8275 - F1: 0.7306 - Weighted_F1: 0.8235 - val_loss: 0.5405 - val_acc: 0.8121 - val_F1: 0.7208 - val_Weighted_F1: 0.8083

Epoch 00003: val_acc improved from 0.79616 to 0.81211, saving model to best_model.h5
Epoch 4/50
497/497 [==============================] - 6s 12ms/step - loss: 0.4536 - acc: 0.8454 - F1: 0.7579 - W

In [34]:
loaded_model = load_model('best_model.h5')

In [35]:
loaded_model.evaluate(X_test_new, y_test_new)

192/192 [==============================] - 1s 5ms/step - loss: 0.5403 - acc: 0.8177 - F1: 0.7155 - Weighted_F1: 0.8113


[0.54034823179245,
 0.817676842212677,
 array([0.91896266, 0.8090979 , 0.88058853, 0.80553424, 0.4488189 ,
        0.61878455, 0.5267176 ], dtype=float32),
 0.8112874031066895]